In [1]:
import os
import time
import random
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from augraphy import *
from sklearn.model_selection import KFold
from glob import glob
import cv2
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [2]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target, img_path

In [3]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets, _ in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return model, ret




def evaluation_one_epoch(loader, model, optimizer, loss_fn, device):
    model.eval()  # 모델을 평가 모드로 설정
    valid_loss = 0.0
    valid_accuracy = 0
    preds_list = []
    targets_list = []

    with torch.no_grad(): # model의 업데이트 막기
        pbar = tqdm(loader)
        for images, targets, _ in pbar:
            images = images.to(device)
            targets = targets.to(device)

            # 순전파
            preds = model(images)
            loss = loss_fn(preds, targets)

            # 손실과 정확도 계산
            valid_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

            pbar.set_description(f"Loss: {loss.item():.4f}")

        valid_loss /= len(loader)
        valid_acc = accuracy_score(targets_list, preds_list)
        valid_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "valid_loss": valid_loss,
        "valid_acc": valid_acc,
        "valid_f1": valid_f1,
    }

    return model, ret


In [4]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'data/aug_1'

# model config
model_name = 'resnet50' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 224
LR = 1e-3
EPOCHS = 1
BATCH_SIZE = 32
num_workers = 0

In [5]:
# augmentation을 위한 transform 코드
#이미 albumentation에서 진행
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size), 
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=224, width=224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [6]:
# Dataset 정의
aug_path = '/data/ephemeral/home/new_aug.csv'


trn_dataset = ImageDataset(
    aug_path,
    "data/aug_2/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "data/sample_submission.csv",
    "data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

23550 3140


In [7]:
df = pd.read_csv(aug_path)

In [9]:
df.head()

,ID,target
0,augmented_0_a1ab865095b2d312_ljh.jpg,2
1,augmented_1_a1ab865095b2d312_ljh.jpg,2
2,augmented_2_a1ab865095b2d312_ljh.jpg,2
3,augmented_3_a1ab865095b2d312_ljh.jpg,2
4,augmented_4_a1ab865095b2d312_ljh.jpg,2


In [10]:
from torch.utils.data import DataLoader, random_split

# 데이터셋 크기 정의
total_size = len(trn_dataset)
train_size = int(total_size * 0.8)  # 예를 들어, 전체 데이터의 80%
val_size = total_size - train_size  # 나머지 20%

# 데이터셋 분할
train_dataset, val_dataset = random_split(trn_dataset, [train_size, val_size])

# DataLoader 정의
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)

val_loader = DataLoader(
    val_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = num_workers, 
    pin_memory = True, drop_last = False
)

test_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)


print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(tst_dataset)}")


Training dataset size: 18840
Validation dataset size: 4710
Test dataset size: 3140


In [11]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [12]:
import torchsummary
torchsummary.summary(model, input_size = (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
         Identity-10           [-1, 64, 56, 56]               0
             ReLU-11           [-1, 64, 56, 56]               0
         Identity-12           [-1, 64, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [14]:
from torch.utils.tensorboard import SummaryWriter


def training_loop(model, train_loader, val_loader, loss_fn, optimizer, device, num_epochs, patience, model_name):
    writer = SummaryWriter(f'runs/{model_name}')  # TensorBoard 요약 작성자 초기화
    best_valid_loss = float('inf')  # 가장 좋은 validation loss를 저장
    early_stop_counter = 0  # 카운터
    valid_max_accuracy = -1
    valid_max_f1 = -1
    best_max_f1 = -1

    for epoch in range(num_epochs):
        model, ret = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        model, ret_val = evaluation_one_epoch(val_loader, model, optimizer, loss_fn, device)

        writer.add_scalar('Loss/train', ret['train_loss'], epoch)
        writer.add_scalar('Accuracy/train', ret['train_acc'], epoch)
        writer.add_scalar('F1/train', ret['train_f1'], epoch)

        # TensorBoard에 검증 지표 기록
        writer.add_scalar('Loss/valid', ret_val['valid_loss'], epoch)
        writer.add_scalar('Accuracy/valid', ret_val['valid_acc'], epoch)
        writer.add_scalar('F1/valid', ret_val['valid_f1'], epoch)

        
        if ret_val['valid_acc'] > valid_max_accuracy:
            valid_max_accuracy = ret_val['valid_acc']

        if ret_val['valid_f1'] < best_max_f1:
            best_max_f1 = ret_val['valid_f1']
        
        # validation loss가 감소하면 모델 저장 및 카운터 리셋
        if ret_val['valid_loss'] < best_valid_loss:
            best_valid_loss = ret_val['valid_loss']
            torch.save(model.state_dict(), f"./model_{model_name}.pt")
            early_stop_counter = 0


        
        # validation loss가 증가하거나 같으면 카운터 증가
        else:
            early_stop_counter += 1

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {ret['train_loss']:.4f}, Train Accuracy: {ret['train_acc']:.4f} Valid Loss: {ret_val['valid_loss']:.4f}, Valid Accuracy: {ret_val['valid_acc']:.4f}")

        # 조기 종료 카운터가 설정한 patience를 초과하면 학습 종료
        if early_stop_counter >= patience:
            print("Early stopping")
            break
    writer.close()
    return model, valid_max_accuracy, ret, ret_val

In [15]:
num_epochs = 100
patience = 20
experiment_name = 'final_resnet50'

model, valid_max_accuracy, ret, ret_val = training_loop(model, train_loader, val_loader, loss_fn, optimizer, device, num_epochs, patience, experiment_name)
# print("resnet34 valid max accuracy: ", valid_max_accuracy)

Loss: 0.1244: 100%|██████████| 148/148 [00:12<00:00, 11.90it/s]


Epoch [1/100], Train Loss: 0.6435, Train Accuracy: 0.7815 Valid Loss: 0.3264, Valid Accuracy: 0.8737


Loss: 0.0261: 100%|██████████| 148/148 [00:11<00:00, 12.55it/s]


Epoch [2/100], Train Loss: 0.1965, Train Accuracy: 0.9276 Valid Loss: 0.2017, Valid Accuracy: 0.9297


Loss: 0.0222: 100%|██████████| 148/148 [00:12<00:00, 11.43it/s]


...

Epoch [58/100], Train Loss: 0.0025, Train Accuracy: 0.9992 Valid Loss: 0.1470, Valid Accuracy: 0.9633


Loss: 0.0799: 100%|██████████| 148/148 [00:10<00:00, 14.24it/s]


Epoch [59/100], Train Loss: 0.0097, Train Accuracy: 0.9966 Valid Loss: 0.1358, Valid Accuracy: 0.9667


Loss: 0.0014: 100%|██████████| 148/148 [00:10<00:00, 14.20it/s]

Epoch [60/100], Train Loss: 0.0081, Train Accuracy: 0.9978 Valid Loss: 0.1267, Valid Accuracy: 0.9671
Early stopping


In [19]:
ret

{'train_loss': 0.008089438010473024,
 'train_acc': 0.9977707006369426,
 'train_f1': 0.9977657645522777}

In [20]:
ret_val

{'valid_loss': 0.12667561614171063,
 'valid_acc': 0.9670912951167728,
 'valid_f1': 0.9649240713950324}

In [16]:
def submission(model, model_name, test_loader):
    preds_list = []
    model.load_state_dict(torch.load(f"./model_{model_name}.pt")) # 모델 불러오기
    model = model.to(device)
    model.eval()    
    for image, _, _ in tqdm(test_loader):
        image = image.to(device)

        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list
    sample_submission_df = pd.read_csv("data/sample_submission.csv")
    assert (sample_submission_df['ID'] == pred_df['ID']).all()
    pred_df.to_csv(f"pred_{model_name}.csv", index=False)
    print(pred_df.head())


In [ ]:
def submission(model, model_name, test_loader):
    preds_list = []
    model.load_state_dict(torch.load(f"./model_{model_name}.pt")) # 모델 불러오기
    model = model.to(device)
    model.eval()    
    for image, _, _ in tqdm(test_loader):
        image = image.to(device)

        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
    pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list
    sample_submission_df = pd.read_csv("data/sample_submission.csv")
    assert (sample_submission_df['ID'] == pred_df['ID']).all()
    pred_df.to_csv(f"pred_{model_name}.csv", index=False)
    print(pred_df.head())


In [18]:
submission(model, "final_resnet50", test_loader)

100%|██████████| 99/99 [00:14<00:00,  6.83it/s]

                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      12
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg      12
4  00901f504008d884.jpg       2
